<a href="https://colab.research.google.com/github/ravi-gopalan/DAND_Data_Wrangling/blob/master/yelp_review_spacy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [2]:
pwd

'/gdrive'

# Import Libraries

In [0]:
import pandas as pd
from pandas.io.json import json_normalize
import json

In [4]:
cd '/gdrive/My Drive/yelp_dataset'

/gdrive/My Drive/yelp_dataset


In [5]:
!ls

business.json				 tip.json
checkin.json				 user.json
Dataset_Challenge_Dataset_Agreement.pdf  Yelp_Dataset_Challenge_Round_13.pdf
photo.json				 yelp_photos
review.json


In [5]:
df_business = pd.read_json('business.json', lines=True)
df_business.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192609 entries, 0 to 192608
Data columns (total 14 columns):
business_id     192609 non-null object
name            192609 non-null object
address         192609 non-null object
city            192609 non-null object
state           192609 non-null object
postal_code     192609 non-null object
latitude        192609 non-null float64
longitude       192609 non-null float64
stars           192609 non-null float64
review_count    192609 non-null int64
is_open         192609 non-null int64
attributes      163773 non-null object
categories      192127 non-null object
hours           147779 non-null object
dtypes: float64(3), int64(2), object(9)
memory usage: 20.6+ MB


In [6]:
df_business.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,1SWheh84yJXfytovILXOAQ,Arizona Biltmore Golf Club,2818 E Camino Acequia Drive,Phoenix,AZ,85016,33.522143,-112.018481,3.0,5,0,{'GoodForKids': 'False'},"Golf, Active Life",None
1,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,30 Eglinton Avenue W,Mississauga,ON,L5R 3E7,43.605499,-79.652289,2.5,128,1,"{'RestaurantsReservations': 'True', 'GoodForMe...","Specialty Food, Restaurants, Dim Sum, Imported...","{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W..."
2,gnKjwL_1w79qoiV3IC_xQQ,Musashi Japanese Restaurant,"10110 Johnston Rd, Ste 15",Charlotte,NC,28210,35.092564,-80.859132,4.0,170,1,"{'GoodForKids': 'True', 'NoiseLevel': 'u'avera...","Sushi Bars, Restaurants, Japanese","{'Monday': '17:30-21:30', 'Wednesday': '17:30-..."
3,xvX2CttrVhyG2z1dFg_0xw,Farmers Insurance - Paul Lorenz,"15655 W Roosevelt St, Ste 237",Goodyear,AZ,85338,33.455613,-112.395596,5.0,3,1,None,"Insurance, Financial Services","{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ..."
4,HhyxOkGAM07SRYtlQ4wMFQ,Queen City Plumbing,"4209 Stuart Andrew Blvd, Ste F",Charlotte,NC,28217,35.190012,-80.887223,4.0,4,1,"{'BusinessAcceptsBitcoin': 'False', 'ByAppoint...","Plumbing, Shopping, Local Services, Home Servi...","{'Monday': '7:0-23:0', 'Tuesday': '7:0-23:0', ..."


In [0]:
# 1 = open, 0 = closed
df_business = df_business[df_business['is_open']==1]

In [0]:
drop_columns = ['hours','is_open','review_count']
df_business = df_business.drop(drop_columns, axis=1)

In [9]:
df_business.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 158525 entries, 1 to 192607
Data columns (total 11 columns):
business_id    158525 non-null object
name           158525 non-null object
address        158525 non-null object
city           158525 non-null object
state          158525 non-null object
postal_code    158525 non-null object
latitude       158525 non-null float64
longitude      158525 non-null float64
stars          158525 non-null float64
attributes     132752 non-null object
categories     158064 non-null object
dtypes: float64(3), object(8)
memory usage: 14.5+ MB


In [10]:
df_explode = df_business.assign(categories = df_business.categories.str.split(', ')).explode('categories')
df_explode.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 659391 entries, 1 to 192607
Data columns (total 11 columns):
business_id    659391 non-null object
name           659391 non-null object
address        659391 non-null object
city           659391 non-null object
state          659391 non-null object
postal_code    659391 non-null object
latitude       659391 non-null float64
longitude      659391 non-null float64
stars          659391 non-null float64
attributes     572312 non-null object
categories     658930 non-null object
dtypes: float64(3), object(8)
memory usage: 60.4+ MB


In [11]:
df_explode.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,attributes,categories
1,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,30 Eglinton Avenue W,Mississauga,ON,L5R 3E7,43.605499,-79.652289,2.5,"{'RestaurantsReservations': 'True', 'GoodForMe...",Specialty Food
1,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,30 Eglinton Avenue W,Mississauga,ON,L5R 3E7,43.605499,-79.652289,2.5,"{'RestaurantsReservations': 'True', 'GoodForMe...",Restaurants
1,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,30 Eglinton Avenue W,Mississauga,ON,L5R 3E7,43.605499,-79.652289,2.5,"{'RestaurantsReservations': 'True', 'GoodForMe...",Dim Sum
1,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,30 Eglinton Avenue W,Mississauga,ON,L5R 3E7,43.605499,-79.652289,2.5,"{'RestaurantsReservations': 'True', 'GoodForMe...",Imported Food
1,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,30 Eglinton Avenue W,Mississauga,ON,L5R 3E7,43.605499,-79.652289,2.5,"{'RestaurantsReservations': 'True', 'GoodForMe...",Food


In [12]:
df_explode.categories.value_counts()

Restaurants                    42237
Shopping                       26734
Food                           23208
Home Services                  18455
Beauty & Spas                  16545
                               ...  
Medical Foot Care                  1
Kiosk                              1
Halfway Houses                     1
Street Art                         1
Sauna Installation & Repair        1
Name: categories, Length: 1290, dtype: int64

In [17]:
# open input file: 
ifile = open('review.json') 

# read the first 100k entries
# set to -1 to process everything
stop = 1000000

all_data = list()
for i, line in enumerate(ifile):
    if i%10000==0:
        print(i)
    if i==stop:
        break    
    # convert the json on this line to a dict
    data = json.loads(line)
    # extract what we want
    text = data['text']
    stars = data['stars']
    # add to the data collected so far
    all_data.append([stars, text])
# create the DataFrame
df_review = pd.DataFrame(all_data, columns=['stars','text'])
print(df_review)
# df.to_hdf('revie20ws.h5','reviews')

ifile.close()

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
        stars                                               text
0         1.0  Total bill for this horrible service? Over $8G...
1         5.0  I *adore* Travis at the Hard Rock's new Kelly ...
2         5.0  I have to say that this office really has it t...
3         5.0  Went in for a lunch. Steak sand

In [15]:
!pip freeze

absl-py==0.8.1
alabaster==0.7.12
albumentations==0.1.12
altair==3.3.0
asgiref==3.2.3
astor==0.8.0
astropy==3.0.5
atari-py==0.2.6
atomicwrites==1.3.0
attrs==19.3.0
audioread==2.1.8
autograd==1.3
Babel==2.7.0
backcall==0.1.0
backports.tempfile==1.0
backports.weakref==1.0.post1
beautifulsoup4==4.6.3
bleach==3.1.0
blis==0.2.4
bokeh==1.0.4
boto==2.49.0
boto3==1.10.32
botocore==1.13.32
Bottleneck==1.3.1
branca==0.3.1
bs4==0.0.1
bz2file==0.98
cachetools==3.1.1
certifi==2019.11.28
cffi==1.13.2
chainer==6.5.0
chardet==3.0.4
chart-studio==1.0.0
Click==7.0
cloudpickle==1.2.2
cmake==3.12.0
colorlover==0.3.0
community==1.0.0b1
contextlib2==0.5.5
convertdate==2.2.0
coverage==3.7.1
coveralls==0.5
crcmod==1.7
cufflinks==0.17.0
cupy-cuda101==6.5.0
cvxopt==1.2.3
cvxpy==1.0.25
cycler==0.10.0
cymem==2.0.3
Cython==0.29.14
daft==0.0.4
dask==1.1.5
dataclasses==0.7
datascience==0.10.6
decorator==4.4.1
defusedxml==0.6.0
descartes==1.1.0
dill==0.3.1.1
distributed==1.25.3
Django==3.0
dlib==19.18.0
dm-sonnet==1.3

In [0]:
# Import the English language class
from spacy.lang.en import English

In [0]:
# Create the nlp object
nlp = English()

In [16]:
# Created by processing a string of text with the nlp object
doc = nlp("Hello world!")

# Iterate over tokens in a Doc
for token in doc:
    print(token.text)

Hello
world
!


In [0]:
import numpy as np
import random
check = random.randint(0,1000000)

In [19]:
check = random.randint(0,1000000)
list(df_review['text'][check:check+10])

['Always impressed regardless by the amount of times I\'ve seen it. \n\nWho doesn\'t love a little Frank Sinatra playing with a bonus of a synchronized water? The amount of room to watch the show is also pretty generous and it is often enough that you can come back or even wait around to catch the next round. It\'s also free to watch and finding anything for free these days can be a challenge.\n\nThe only reason for the 4 instead of 5 is due to the fact that I had the opportunity to watch "World of Color" at Disneyland and that is THE MOST AMAZING water show in the world (of the 4 or 5 that I\'ve seen anyway!).',
 "Great place to stop in between gambling as the restaurant is located in MGM. The food was surprisingly really good as I understand that this is a chain restaurant. \n\nIt has a nice ambient feel to this place and the wait staff was attentive and helpful.   \n\nI ordered steak skewers as appetizer. The beef was tender and juicy. \n\nFor dessert, I ordered the chocolate souffl

In [12]:
import spacy.cli
spacy.cli.download("en_core_web_lg")
import en_core_web_lg
nlp = en_core_web_lg.load()

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [13]:
import spacy
# Load a larger model with vectors
nlp = spacy.load('en_core_web_lg')

# Compare two documents
doc1 = nlp("I like fast food")
doc2 = nlp("I like pizza")
print(doc1.similarity(doc2))

0.8627204117787385


In [23]:
check = random.randint(0,1000000)
list(df_review['text'][check:check+10])

["When you walk in they watch you as though you're going to steal something. I have gone in a couple times with friends and family and there is no winning. Clothes aren't nice enough to be treated that way and honestly I rather invest in people who are genuinely interested rather than poor snobs. Should go out of business soon.",
 "The strangest thing about Brio is that it's setting it looks like a movie set, without any extras. A pristine street with a fountain, perfect brick streets but empty as all hell. \n\nIt was lunch, we were starving so we stumbled in. They seated us quickly, the ambiance was pleasant but not note worthy, we outside in our disturbingly quiet area and found it quite pleasant. Ladies who lunch, dripping with jewels and puffy clouds passing by, then it came down to the food. \n\nBrio is a great place for lunch for people who have already eaten lunch because the portions are small. \n\nI had crab and shrimp cakes that were miniature in size at best, with a side of 

In [0]:
# Created by processing a string of text with the nlp object
check = random.randint(0,1000000)
docs = list(df_review['text'][check:check+10])
nlp_list = []
for item in docs:
  nlp_list.append(nlp(item))


s = (len(docs),len(docs))
similarity = np.zeros(s)


In [40]:
check = random.randint(0,1000000)
print(check)
list(df_review['text'][check:check+10])


196985


["800 Degree's just opened, so have to give them some time to get into the groove.  The service while friendly is very disorganized.  Ordered salads and pizza, and the pizza was done first.  This is in spite of having 8 staff literally standing around (there were only 6 people in the place).  The Pizza was nothing like the Westwood, CA main location. Sauce concentrated in center, so overly wet and could not lift up the pizza and eat easily, and could not keep toppings on pizza.  The Burrata was also a bit runny compared to the usual quality from the original.\n\nManagement and expeditors need to work on staff training if they want to preserve the unique and fabulous experience that keeps the CA original with lines out the door.",
 'For starters, this is a SPORTS bar.  If you want just a BAR, go elsewhere.  Also, this is mainly an "Eagles" football place.  If you want a team agnostic place, go elsewhere.  I experienced this on a Sunday that the Eagles were to play on Monday night.  If m

In [42]:
similarity

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [43]:
for i in range(len(docs)):
  for j in range(len(docs)):
    similarity[i][j] = nlp_list[i].similarity(nlp_list[j])
similarity

array([[1.        , 0.9740605 , 0.86304307, 0.98156664, 0.95036943,
        0.94708729, 0.95854179, 0.97418125, 0.95381807, 0.95352025],
       [0.9740605 , 1.        , 0.88528665, 0.98066592, 0.95568925,
        0.96076591, 0.94500574, 0.96698724, 0.95904372, 0.96898236],
       [0.86304307, 0.88528665, 1.        , 0.86475462, 0.8998834 ,
        0.91926741, 0.86499728, 0.84800727, 0.9279712 , 0.91011588],
       [0.98156664, 0.98066592, 0.86475462, 1.        , 0.95150767,
        0.94945884, 0.96319252, 0.97007998, 0.9506452 , 0.95382896],
       [0.95036943, 0.95568925, 0.8998834 , 0.95150767, 1.        ,
        0.9383397 , 0.93615731, 0.93503038, 0.9459369 , 0.94921302],
       [0.94708729, 0.96076591, 0.91926741, 0.94945884, 0.9383397 ,
        1.        , 0.93914946, 0.93386524, 0.97708247, 0.97728992],
       [0.95854179, 0.94500574, 0.86499728, 0.96319252, 0.93615731,
        0.93914946, 1.        , 0.95094043, 0.95023271, 0.93095181],
       [0.97418125, 0.96698724, 0.8480072

In [50]:
print(docs[9])
print(docs[similarity[9][:-1].argmax()+1])


This place has really gone downhill. A few years back when it had first opened it was a great place for decently priced Japanese/Korean food. Came here recently and found the quality has decreased. 

There's no parking in the front of the plaza where Gal's is, you will have to park in the main parking area or underground and walk around. The decor is a typical Japanese restaurant though I will say there are lots of tables and the restaurant is always lively. There are decently priced lunch and dinner combos and will come with Korean glass noodles, rice, soup and salad. 

The sashimi in my combo today was thinly sliced with little to no fish flavour.  Texture felt a bit odd so I ended up not eating  all of it. The sushi looked like it was falling apart.  The plates and soup bowls are lined red interior and the ink/paint is peeling off badly. Not only does this make it look dated, I also feel like the paint might go into my food. Please replace your bowls and plates.
This State Farm is p

In [20]:
# A slice from the Doc is a Span object
span = doc[1:4]

# Get the span text via the .text attribute
print(span.text)

worst sushi we


In [22]:
doc = nlp("It costs $5.")
print('Index:   ', [token.i for token in doc])
print('Text:    ', [token.text for token in doc])
print('is_alpha:', [token.is_alpha for token in doc])
print('is_punct:', [token.is_punct for token in doc])
print('like_num:', [token.like_num for token in doc])

Index:    [0, 1, 2, 3, 4]
Text:     ['It', 'costs', '$', '5', '.']
is_alpha: [True, True, False, False, False]
is_punct: [False, False, False, False, True]
like_num: [False, False, False, True, False]


In [23]:
check = random.randint(0,1000000)
doc = nlp(df_review['text'][check])
print('Index:   ', [token.i for token in doc])
print('Text:    ', [token.text for token in doc])
print('is_alpha:', [token.is_alpha for token in doc])
print('is_punct:', [token.is_punct for token in doc])
print('like_num:', [token.like_num for token in doc])

Index:    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219,

In [24]:
import spacy


Tots

truffle honey mustard - yum!
Tots NOUN compound yum


 SPACE  Tots
truffle VERB amod yum
honey NOUN compound yum
mustard NOUN compound yum
- PUNCT punct yum
yum NOUN ROOT yum
! PUNCT punct yum


In [25]:
spacy.explain('ORG')

'Companies, agencies, institutions, etc.'

In [34]:
spacy.explain('CARDINAL')

'Numerals that do not fall under another type'

In [35]:
spacy.explain('CCONJ')

'coordinating conjunction'

In [26]:
nlp = spacy.load('en_core_web_sm')

check = random.randint(0,1000000)
doc = nlp(df_review['text'][check])

print(df_review['text'][check])
# Iterate over the tokens
for token in doc:
    # Print the text and the predicted part-of-speech tag
    print(token.text, token.pos_, token.dep_, token.head.text)

# Iterate over the predicted entities
for ent in doc.ents:
    # Print the entity text and its label
    print(ent.text, ent.label_)

This place has great burgers and they offer gluten free buns! It's also a fun place to watch sports games.
This DET det place
place NOUN nsubj has
has VERB ROOT has
great ADJ amod burgers
burgers NOUN dobj has
and CCONJ cc has
they PRON nsubj offer
offer VERB conj has
gluten VERB amod buns
free ADJ amod buns
buns NOUN dobj offer
! PUNCT punct offer
It PRON nsubj 's
's VERB ROOT 's
also ADV advmod 's
a DET det place
fun ADJ compound place
place NOUN attr 's
to PART aux watch
watch VERB relcl place
sports NOUN compound games
games NOUN dobj watch
. PUNCT punct 's
